In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
import numpy as np

# 1) 读数据
df = pd.read_csv(r'C:\Users\khand\OneDrive\code\aco\ACO\method\V_all.csv')


In [57]:
feature_names = ['LLs', 'p1', 'd1', 'p2', 'd2', 'p3', 'd3', 'p4', 'd4', 'p5', 'd5',
                 'p6', 'd6', 'p7', 'd7', 'p8', 'LLe', 'R1', 'R2', 'k']
mask = ['d2', 'p3', 'd3','p4', 'd6', 'p7','d7','p8', 'R1', 'R2', 'label']
mask2 = ['p1', 'd1', 'p2', 'd2', 'p3', 'd3', 'p4', 'd4', 'p5', 'd5',
        'p6', 'd6', 'p7', 'd7', 'p8', 'R1', 'R2', 'k','label']
y = df["label"]
tmp = df.loc[:,mask2]
tmp

,p1,d1,p2,d2,p3,d3,p4,d4,p5,d5,p6,d6,p7,d7,p8,R1,R2,k,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,1
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,1
4,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59955,0,0,1,2,0,2,0,0,2,0,1,0,0,0,0,0,0,1,0
59956,1,1,2,0,1,0,0,1,2,0,1,1,0,1,0,0,0,0,0
59957,0,1,2,0,1,0,0,1,2,0,1,1,0,1,0,0,0,0,0
59958,1,0,2,0,1,0,0,1,2,0,1,1,0,1,0,1,0,0,0


In [107]:
condition0 = df['label'] == 0
dead = df[condition0]
live = df[~condition0]
condition1 = (dead['p3'] == 1).values * (dead['R2'] == 0).values * (dead['R1'] == 0).values
X1 = dead[condition1]
X2 = dead[~condition1]
X1.var()

LLs      2.017454
p1       0.249648
d1       0.616387
p2       0.451709
d2       0.587222
p3       0.000000
d3       0.184150
p4       0.249794
d4       0.342834
p5       0.725951
d5       0.245852
p6       0.241415
d6       0.540650
p7       0.000000
d7       0.225813
p8       0.250085
LLe      1.886495
R1       0.000000
R2       0.000000
k        0.000000
label    0.000000
dtype: float64

In [ ]:
y = np.concat((np.ones(len(X1)),2 * np.ones(len(X2)),np.zeros(len(live))))
y = pd.DataFrame(y)
X = pd.concat((X1,X2,live), axis=0)
X.drop('label',axis=1, inplace=True)
print(X.shape,y.shape)
#print(X.var())
print(X.columns)

In [ ]:
total = dead.shape[0]
print("total:",total)
dead = dead[dead['p3'] == 1 and dead['R2'] == 0]
print("p3=1 and 'R2' == 0           ",dead.shape[0])
var = dead.var()
var = var[var > 0.01]
print(var)

In [36]:
tr1 = [0,16]
tr2 = [0, 1, 2, 3, 7, 8, 9, 10, 11, 15, 16]
tr3 = [0, 1, 2, 7, 8, 9, 10, 11, 16]
#df = df[mask]
print("n_sample:",df.shape[0])
df = df.drop_duplicates()
X = df.iloc[:, :-1]
y = df['label']
print("n_sample:",X.shape[0])

n_sample: 59960
n_sample: 59862


In [101]:
# 3) 划分训练/测试（按类别分层）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y, shuffle=True
)

# 4) 训练随机森林（更稳健的参数）
clf = RandomForestClassifier(
    n_estimators=300,          # 树的数量
    max_depth=10,              # 最大深度（防止过拟合）
    class_weight='balanced',   # 类别权重平衡
    n_jobs=-1,                 # 使用所有CPU核心加速训练
    random_state=42
)
clf.fit(X_train, y_train)

# 评估
y_pred = clf.predict(X_test)

# 输出分类报告和 ROC-AUC
print("\n", classification_report(y_test, y_pred, digits=4))

# 特征重要性
importances = clf.feature_importances_
imp_order = np.argsort(importances)[::-1]  # 按照重要性降序排列

# 特征名称（根据原数据集调整特征名称）
feature_names = ['LLs', 'p1', 'd1', 'p2', 'd2', 'p3', 'd3', 'p4', 'd4', 'p5', 'd5',
                 'p6', 'd6', 'p7', 'd7', 'p8', 'LLe', 'R1', 'R2', 'k']

print("\nTop-3 feature importances:")
for i in imp_order[:5]:
    print(f"{X.columns[i]}: {importances[i]:.2f}")

D:\anaconda\envs\dqn\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



               precision    recall  f1-score   support

         0.0     0.9993    0.9512    0.9746     17291
         1.0     0.4306    0.9848    0.5992       394
         2.0     0.4729    0.9802    0.6380       303

    accuracy                         0.9524     17988
   macro avg     0.6343    0.9721    0.7373     17988
weighted avg     0.9779    0.9524    0.9607     17988


Top-3 feature importances:
p3: 0.22
R2: 0.14
p5: 0.10
p7: 0.07
k: 0.07
